In [4]:
import pandas as pd
import numpy as np
import json
import requests
import xmltodict
import time
import pickle
import os
import sqlalchemy
import urllib.request
from mutagen.mp3 import MP3

In [5]:
config = json.load(open('../config.json', 'r'))

In [6]:
user = config['db']['username']
password = config['db']['password']
host = config['db']['host']
port = config['db']['port']
dbname = config['db']['dbname']

In [7]:
template = "mysql+pymysql://{user}:{password}@{host}:{port}/{dbname}"
connection_string = template.format(user=user, password=password, host=host, port=port, dbname=dbname)

In [8]:
conn = sqlalchemy.create_engine(connection_string, pool_size=1)

In [21]:
if os.path.isfile(fname):
    os.remove(fname)

In [22]:
fname = 'feed.xml'
url = 'http://dataskeptic.com/feed.rss'

if not(os.path.isfile(fname)):
    print('fetching')
    r = requests.get(url)
    f = open(fname, 'wb')
    f.write(r.text.encode('utf-8'))
    f.close()

with open(fname) as fd:
    xml = xmltodict.parse(fd.read())

fetching


In [23]:
emap = {}
episodes = xml['rss']['channel']['item']
for episode in episodes:
    guid = episode['guid']['#text']
    url = episode['enclosure']['@url']
    emap[guid] = url

In [31]:
q = """
SELECT t1.blog_id, prettyname, guid
 , t2.content_id as mp3_content, t3.content_id as homepage_content
 , t1.title, t1.abstract
FROM blog t1
LEFT JOIN related_content t2
 on t1.blog_id = t2.blog_id
 and t2.type='mp3'
LEFT JOIN related_content t3
 on t1.blog_id = t3.blog_id
 and t3.type='homepage-image'
WHERE t1.prettyname like '/episodes/%%'
AND (t2.blog_id is NULL or t3.blog_id is null)
LIMIT 10
"""
df = pd.read_sql(q, conn)

In [32]:
df.shape

(1, 7)

In [28]:
tpl = """
INSERT INTO related_content (blog_id, dest, type, title, body, duration) VALUES 
({blog_id}, '{dest}', '{type}', '{title}', '{body}', {duration})
"""

In [34]:
dest

'http://traffic.libsyn.com/dataskeptic/ai-in-games-master.mp3?dest-id=201630'

In [35]:
for r in range(df.shape[0]):
    row = df.iloc[r]
    blog_id = row['blog_id']
    guid = row['guid']
    title = row['title'].replace("'", "\\'")
    body = row['abstract'].replace("'", "\\'")
    dest = emap[guid]
    response = urllib.request.urlopen(dest)
    data = response.read()
    fname = 'temp.mp3'
    f = open(fname, 'wb')
    f.write(data)
    f.close()
    audio = MP3(fname)
    duration = int(audio.info.length)
    q1 = tpl.format(blog_id=blog_id, dest=dest, type='mp3', title=title, body=body, duration=duration)
    r1 = conn.execute(q1)
    dest = 'https://s3.amazonaws.com/dataskeptic.com/img/2018/ai-cover.png'
    q2 = tpl.format(blog_id=blog_id, dest=dest, type='homepage-image', title=title, body=body, duration=-1)
    r2 = conn.execute(q2)

In [54]:
#conn.execute("delete from related_content where blog_id=382")

In [36]:
pd.read_sql("SELECT * from related_content order by content_id desc limit 10", conn)

,content_id,blog_id,dest,type,title,body,created_date,blog_id2,duration
0,664,385,https://s3.amazonaws.com/dataskeptic.com/img/2...,homepage-image,AI in Games,Today's interview is with the authors of the t...,2018-05-18 13:09:38,NaN,-1.0
1,663,385,http://traffic.libsyn.com/dataskeptic/ai-in-ga...,mp3,AI in Games,Today's interview is with the authors of the t...,2018-05-18 13:09:35,NaN,1557.0
2,662,384,https://s3.amazonaws.com/dataskeptic.com/img/2...,homepage-image,Game Theory,This week's episode is a short primer on game ...,2018-05-11 18:38:35,NaN,-1.0
3,661,384,http://traffic.libsyn.com/dataskeptic/game-the...,mp3,Game Theory,This week's episode is a short primer on game ...,2018-05-11 18:38:34,NaN,1450.0
4,660,383,https://s3.amazonaws.com/dataskeptic.com/img/2...,homepage-image,Ethical AI at Microsoft Build 2018,,2018-05-07 17:31:12,-1.0,NaN
5,659,382,https://s3.amazonaws.com/dataskeptic.com/img/2...,homepage-image,The Experimental Design of Paranormal Claims,"In this episode of Data Skeptic, Kyle chats wi...",2018-05-04 16:36:31,NaN,-1.0
6,658,382,http://traffic.libsyn.com/dataskeptic/the-expe...,mp3,The Experimental Design of Paranormal Claims,"In this episode of Data Skeptic, Kyle chats wi...",2018-05-04 16:36:31,NaN,1651.0
7,653,370,,internal-link,Part 2,Listen to the next segment!,2018-05-03 00:03:34,371.0,NaN
8,652,380,https://s3.amazonaws.com/dataskeptic.com/guest...,person,Hector Levesque,Hector Levesque holds a PhD in computer scienc...,2018-04-30 03:17:17,-1.0,NaN
9,651,381,https://s3.amazonaws.com/dataskeptic.com/img/2...,homepage-image,Table tennis robot trained with reinforcement ...,,2018-04-30 01:57:53,-1.0,NaN
